# PROJET DATA


## Contexte

Depuis les années 90, il y a eu une véritable prise de conscience mondiale de la nécessité de réduire la consommation d’énergie et des émissions de gaz à effet de serre. Les premiers engagements sont apparus lors de la signature du protocole de Kyoto en 1997. Mais son entrée en vigueur n’a finalement eu lieu qu’en 2005 et de nombreux scientifiques ont jugé les efforts insuffisants pour ralentir le réchauffement climatique. Depuis, d’autres engagements plus ambitieux ont vus le jour (division par 4 des émissions d’ici 2050 pour la France par exemple, engagements de certaines grandes villes comme Paris). Mais la tâche est compliquée. Les pouvoirs publics et les collectivités territoriales n’ont pas la possibilité d’obliger les entreprises et les particuliers à changer leurs habitudes pour atteindre ces objectifs. L’action se porte donc avant tout à faire évoluer les comportements. L’économie et le recyclage des matières premières, l’amélioration des modes de transports et des performances énergétiques des bâtiments doivent devenir des priorités.

Dans ce sens, L’ADEME (Agence de l’Environnement et de la Maîtrise de l’Energie) a récemment lancé un appel à manifestation d’intérêt pour promouvoir la réalisation de démonstrateurs et d’expérimentations de nouvelles solutions de mobilité pour les personnes et les marchandises adaptées à différents types de territoires. Votre structure CesiCDP est déjà bien implantée dans le domaine. Aidé de nombreux partenaires, vous avez réalisé plusieurs études sur le thème de la Mobilité Multimodale Intelligente. Les nouvelles technologies de transport, plus économiques et moins polluantes ne sont pas sans poser de nouveaux défis notamment d’un point de vue de l’optimisation de la gestion des ressources. Mais ces problèmes de logistique du transport présentent un enjeu majeur pour l’avenir : ses applications sont nombreuses (distribution du courrier, livraison de produits, traitement du réseau routier, ramassage des ordures) et leur impact sur l’environnement peut être véritablement significatif. Votre étude s’inscrit donc dans le cadre d’une réponse à l’appel de l’ADEME.


## Sujet

Le but de votre étude est de générer une tournée de livraison (problème du VRP). Le problème algorithmique consiste donc à calculer sur un réseau routier une tournée permettant de relier entre elles un sous-ensemble de villes, puis de revenir à son point de départ, de manière à minimiser la distance totale parcourue.

Vous devrez proposer une méthode issue de la Recherche Opérationnelle pour générer une tournée de livraison correspondant à ce problème. L’implémentation se fera sur une version de base du problème, à laquelle vous pourrez ajouter des contraintes supplémentaires, rendant le problème plus réaliste, mais aussi plus dur à traiter.

Par ailleurs, vous devrez effectuer une étude statistique du comportement de votre méthode de résolution, faisant apparaitre ses performances (qualité de solution, temps de convergence). Idéalement, des statistiques prédictives permettent d’extrapoler ce comportement sur des cas d’usages que vos ordinateurs seuls ne pourraient traiter.

## Spécificités techniques

Dans le cas de notre résolution, nous avons décidé de mettre en place les contraintes suivantes dans le cas du sujet :

*   Le graphique est constitué d'un nombre donné de nodes (qui représenteront les villes)
*   Chaque ville attend un nombre aléatoire de produits
*  Le nombre de camions disponibles est défini par l'utilisateur
*  Les camions ont une capacité de stockage définie par l'utilisateur
*  Tous les camions partent de la même ville
*  La distance entre les villes est définies par leur position géographique



# Présentation de la solution

## Bibliothéques Python

Pour répondre à nos besoins dans le cadre du développement de la solution, nous avons eu recours à plusieurs bibliothéques python.

### Google OR-Tools

Pour la mise en place de la solution, nous avons choisi d'utiliser bibliothéque Google OR-Tools, qui permet d'implémenter plusieurs algorithmes de résolution, voici son implémentation :


In [0]:
# Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


Ici, on peut voir que l'on choisir le dataset qui va être utilisé, ainsi que le type d'algorithme qui va être implémenter (avec les particularités s'il est heuristique ou méta-heuristique)

## Description des algorithmes

Dans la résolution de notre de problème, nous avons décidé de choisir 3 algorithmes de résolution de problème pour répondre à la problèmatique. Le but de ce document est donc de faire une description de ces algorithmes, de présenter leurs spécificités, ainsi que les résultats que l'on a obtenu lors de leur utilisation.


### Algorithme de recherche locale

La recherche locale est une méthode générale utilisée pour résoudre des problèmes d'optimisation, c'est-à-dire des problèmes où l'on cherche la meilleure solution dans un ensemble de solutions candidates. La recherche locale consiste à passer d'une solution à une autre solution proche dans l'espace des solutions candidates (l'espace de recherche) jusqu'à ce qu'une solution considérée comme optimale soit trouvée, ou que le temps imparti soit dépassé.

#### Implémentation

Voici le schéma de fonctionnement de l'algorithme :
![recherche locale](http://www.tony-lambert.fr/these/Images/ls4.png)

####Solution
Voici la représentation graphique de la solution de cet algorithme, pour plus de lisibilité nous avons supprimé les liens vers le dépot
![GARL](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/graphs_no_depot/path_cheapest_arc.png)


### Algorithme de recherche tabou

L'idée de la recherche tabou consiste, à partir d'une position donnée, à en explorer le voisinage et à choisir la position dans ce voisinage qui minimise la fonction objectif.

Il est essentiel de noter que cette opération peut conduire à augmenter la valeur de la fonction (dans un problème de minimisation) : c'est le cas lorsque tous les points du voisinage ont une valeur plus élevée. C'est à partir de ce mécanisme que l'on sort d'un minimum local.

Le risque cependant est qu'à l'étape suivante, on retombe dans le minimum local auquel on vient d'échapper. C'est pourquoi il faut que l'heuristique ait de la mémoire : le mécanisme consiste à interdire (d'où le nom de tabou) de revenir sur les dernières positions explorées.

Les positions déjà explorées sont conservées dans une file FIFO (appelée souvent liste tabou) d'une taille donnée, qui est un paramètre ajustable de l'heuristique. Cette file doit conserver des positions complètes, ce qui dans certains types de problèmes, peut nécessiter l'archivage d'une grande quantité d'informations. Cette difficulté peut être contournée en ne gardant en mémoire que les mouvements précédents, associés à la valeur de la fonction à minimiser.

#### Implémentation

Voici le schéma de l'implémentation de l'algorithme :
![recherche tabout](https://slideplayer.fr/slide/1752884/7/images/6/2.+Algorithme+de+recherche+tabou.jpg)

#### Solution
![GTR](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/graphs_no_depot/tabu_search.png)

### Algorithme du gradient

L'algorithme du gradient est également connu sous le nom d'algorithme de la plus forte pente ou de la plus profonde descente (steepest descent, en anglais) parce que le gradient est la pente de la fonction linéarisée au point courant et est donc, localement, sa plus forte pente (notion qui dépend du produit scalaire).

#### Implémentation


Voici le schéma de l'implémentation de l'algorithme :
![gradient descendant](https://www.charlesbordet.com/assets/images/gradient-descent/gradientdescent-f3.gif)

#### Solution 
![ADGG](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/graphs_no_depot/path_cheapest_arc.png)

### Etude de la complexité

Le diagramme suivant présente la compléxité de nos différents algorithmes :

![complexite](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/complexity.png)

On retrouve alors 2 cas :
* Le cas de l'algorithme heuristique (PATH_CHEAPEST_ARC) qui dispose de tout le temps qu'il désire pour trouver une solution qui sera donc plus précise. , On va pouvoir par la suite dégager une courbe représentation de la compléxité, qui sera utilisable dans le cas où l'on va vouloir prédire le comportement de l'algorithme
* Les cas des algorithmes méta-heuristiques ( et ). En effet, de part leur nature méta-heuristique, nous sommes obligés d'imposer un temps limite à l'exécution de l'agoithme, ce pourquoi l'on va retrouver un temps d'exécution constant, quel que soit la taille du problème passé en paramétre.

## Etude Statistique

Dans cette partie seront présentées les différentes études statistiques qui ont été réalisées. Elles seront sous forme de graphiques, qui seront décrits et interprétés.

## Uniformité
L'uniformité montrera la stabilité du temps d'execution selon un même dataset, en effet pour une configuration  donné par exemple avec n villes, n camions, n packet p des dataset seront générés avec des distances entre les villes différentes puis l'algorithme sera executé sur chacun des ces datasets ainsi créés pour en sortir l'uniformité.

On peut uniquement mesurer l'algorithme heuristique, étant donné que les méta-heuristique s'arrête à la limite imposé.

visualisation avec 100 villes :
(axe des ordonnées correspond au nombre d'itération)

![Uniformité_100](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/uniformity/path_cheapest_arc_100.png)

Visualisation avec 500 Villes :

![Uniformité_500](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/uniformity/path_cheapest_arc_500.png)

On constate donc que pour peu de ville le temps d'execution est chaotique, cependant plus le nombre de ville augmente plus l'uniformité est prédictible.

# Statistiques descriptives

#Evolution des variables selon le nombre de villes
Nous allons représenter l'évolution des variables suivante : nombre de camions, temps d'exécution et la distance parcourues
cette représentation est sous forme de nuage de points.

## Temps d'execution
Nous rappelons que pour les méta-heurisque atant donné leur cap en temps de recherche imposé, il n'est pas pertinent de le représenter.
Voici la représentation pour l'heuristique :
![executiontimeScatter](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/scatter_plot/execution_time/path_cheapest_arc.png)
Chaque points rouge corresponds a une execution de l'algorithme sur un même dataset (comme pour l'uniformité) et la courbe bleu représente une fonction d'adoucissement sur la moyenne des points.
Nous pouvons observer que plus le nombre de ville augmente plus le temps d'execution augmente et ainsi via la courbe prédir l'évolution du temps d'execution qui est comme vue ci-dessus ext extrêmement croissant à partir de 1000 villes. Nous recommandons encore une fois l'utilisation d'une méta-heuristique.

##Distance parcourue par les camions et nombre de camions utilisés
Dans cette section verrons qu'il est intéressant de comparer l'évolution de la distance parcourue par les camions dans un dataset donné pour chaque algorithme qu'il soit heuristique ou méta-heuristique

### Algorithme de recherche locale


![ARLD](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/scatter_plot/distance/guided_local_search.png) 
*Distance*
![azer](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/scatter_plot/trucks/guided_local_search.png)
*Nombre de camions*

Dans ce cas on peu observer que la distance parcourue est relative au nombre de villes générée, or contrairement à ce qui est présenté sur la courbe on peu suposer que la croissance exponantielle à partir de l'interprétation mathématique peut être fausse (besoin de plus de calcul)

### Algorithme  de recherche tabou

![ART](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/scatter_plot/distance/tabu_search.png)
*Distance*
![bla](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/scatter_plot/trucks/tabu_search.png)
*Nombre de camions*

Dans ce cas on peu observer que la distance parcourue est relative au nombre de villes générée, or contrairement à ce qui est présenté sur la courbe on peu suposer que la croissance exponantielle à partir de l'interprétation mathématique peut être fausse (besoin de plus de calcul)

En compararant avec l'algorithme de recherche locale on peu voir que les courbes sont similaire. On peu donc déduire que cette statistique descriptive est relative à la nature méta-heuristique de l'algorithme.

###Algorithme du gradient

![AG](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/scatter_plot/distance/path_cheapest_arc.png)
*Distance*
![dqs](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/scatter_plot/trucks/guided_local_search.png)
*Nombre de camions*

Dans ce cas on peu observer que la distance parcourue est relative au nombre de villes générée, or contrairement à ce qui est présenté sur la courbe on peu suposer que la croissance exponantielle à partir de l'interprétation mathématique peut être fausse (besoin de plus de calcul)

Dans ce cas-ci l'algorithme utilisé est heuristique, de ce fait l'hypothèse précédente est fausse, en effet on retrouve à nouveau les même similitude que les deux précédent algorithmes (tendance de la courbe). On peu donc en déduire que celle-ci n'est pas relative à la nature de l'algorithme, on pourra donc supposer de nouvelles hypothèses dans le cas d'une analyse plus poussée.

## Comparaison des algorithmes

L'histogramme de qualité :

L'histogramme de qualité nous permet de visualiser intuitivement la performance de chaque algorithme, en effet nous allons pouvoir voir la différence entre le temps d'execution, le nombre de camion utilisé ainsi que la distance total parcourues par les camions ainsi qu'un ratio représetant la qualité et qui se calcule comme suit :

$qualité = (\frac{distance_{parcourue}}{distance_{arrêtes}}) \times (\frac{camions_{utilisés}}{camions_{disponibles}})$


![quality](https://raw.githubusercontent.com/Exia-Aix-2016/Data-Project/master/screenshots/chart_quality.png)

En rouge : Algorithme de recherche locale ; en vert : l'algorithme de recherche tabou ; en bleu l'algorithme du gradient.

On peut voir que les deux algorithmes méta-heuristique, malgré un temps d'execution limité on une variation de qualité assez faible, on peut donc en déduire que sur de très gros dataset, l'utilisation d'une métat-heuristique est recommandée pour une application industrielle (Important nombre de ville important, une grande flotte de véhicule et un besoin d'exécution rapide).



# CONCLUSION
D'après nos études poussées on peut en conclure  les éléments suivant :
* L'utilisation d'algorithme heuristique est pertinente dans le cas de jeux de données petit (Moins de 1000 villes ou noeuds).
* L'utilisation des algorithmes méta-heuristique est pertinente dans le cas de jeux de données grans (plus de 1000 villes ou noeuds).
* L'étude de complexité n'est pas relatives à la nature de l'algorithme.
* L'étude de la qualité a permis de démontrer que dans un dataset donné, méta-heuristiques et heuristiques ont des caractèristiques d'execution similiraire (Sauf le temps d'exécution).

On peu donc suposer que les meilleurs algorithmes reposent sur un calcule en amont par l'utilisation d'heuristiques qui réalise un pré-traitement sur les donnés qui sont ensuite analysable pour les méta-heuristique.